In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models

In [63]:
search = torch.randn(1, 3, 255, 255)
tample = torch.randn(1, 3, 127, 127)

In [64]:
IN_CHANNELS = 3

In [65]:
def Correlation_func(x, kernel):
    x = x.view(1, -1, x.size(2), x.size(3))  # 1 * (b*c) * k * k
    kernel = kernel.view(-1, 1, kernel.size(2), kernel.size(3))  # (b*c) * 1 * H * W
    out = F.conv2d(x, kernel, groups=x.size(1))
    out = out.view(1, x.size(1), out.size(2), out.size(3))
    return out

In [82]:
base_model = models.resnet50(pretrained=True)
base_layers = list(base_model.children())

class Feat_extractor(nn.Module):
    def __init__(self, out_channels=256):
        super(Feat_extractor, self).__init__()
        self.backbone_list = base_layers[0:7]
        self.search_bb = nn.Sequential(*self.backbone_list[0:7], 
                                       nn.Conv2d(1024, out_channels, kernel_size=1, bias=False)
                                       )
        self.kernel_bb = nn.Sequential(*self.backbone_list[0:7],
                                       nn.Conv2d(1024, out_channels, kernel_size=1, bias=False)
                                       )
        
    def forward(self, search, kernel):
        search = self.convrelu1(search)
        kernel = self.convrelu1(kernel)
        return search, kernel
        

In [83]:
model = Feat_extractor()